# SD202 TP3 - Views, Updates and Database Design

*Author: Benjamin Ternot*

The objectives for this TP are:

1. Create and use SQL Views
2. Update database content
3. Design the database schema for a marketplace


For the first 2 parts we will use the _wine_ database and the Tables created in TP2.

A reminder of the wine database schema:

In [1]:
import sqlite3

In [2]:
def printSchema(connection):
    ### Source: http://stackoverflow.com/a/35092773/4765776
    for (tableName,) in connection.execute(
        """
        select NAME from SQLITE_MASTER where TYPE='table' order by NAME;
        """
    ):
        print("{}:".format(tableName))
        for (
            columnID, columnName, columnType,
            columnNotNull, columnDefault, columnPK,
        ) in connection.execute("pragma table_info('{}');".format(tableName)):
            print("  {id}: {name}({type}){null}{default}{pk}".format(
                id=columnID,
                name=columnName,
                type=columnType,
                null=" not null" if columnNotNull else "",
                default=" [{}]".format(columnDefault) if columnDefault else "",
                pk=" *{}".format(columnPK) if columnPK else "",
            ))

In [3]:
conn = sqlite3.connect('wine.db')
c = conn.cursor()
print("Database schema:")
printSchema(conn)

Database schema:
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)


Again, we recommend inline %sql as an alternative to the sqlite3 package

In [4]:
%load_ext sql
%sql sqlite:///wine.db

Recreate the Tables in 3NF/BCNF from Master2 as you did in the TP2

In [5]:
# Write corresponding code here
%sql DROP TABLE IF EXISTS WINES1;
%sql DROP TABLE IF EXISTS PRODUCERS;
%sql DROP TABLE IF EXISTS PRODUCTIONS;


# Create WINES1 table
%sql CREATE TABLE WINES1 AS \
SELECT DISTINCT NV, CRU, DEGRE, MILL \
FROM MASTER1;

# Create PRODUCERS table
%sql CREATE TABLE PRODUCERS AS \
SELECT DISTINCT NP, NOM, PRENOM, REGION \
FROM MASTER1;

# Create PRODUCTIONS table
%sql CREATE TABLE PRODUCTIONS AS \
SELECT DISTINCT NV, NP, QTE \
FROM MASTER1;


print("\nContent of the database")
printSchema(conn)

# Write corresponding code here
%sql DROP TABLE IF EXISTS WINES2;
%sql DROP TABLE IF EXISTS CLIENTS;
%sql DROP TABLE IF EXISTS SALES;
%sql DROP TABLE IF EXISTS LOC;


# Create WINES2 table
%sql CREATE TABLE WINES2 AS \
SELECT DISTINCT NV, CRU, DEGRE, MILL \
FROM MASTER2;

# Create CLIENTS table
%sql CREATE TABLE CLIENTS AS \
SELECT DISTINCT NB, NOM, PRENOM, TYPE \
FROM MASTER2;

# Create SALES table
%sql CREATE TABLE SALES AS \
SELECT DISTINCT NV, NB, QTE, DATES, LIEU \
FROM MASTER2;

# Create LOC table
%sql CREATE TABLE LOC AS \
SELECT DISTINCT LIEU, REGION \
FROM MASTER2;


print("\nContent of the database")
printSchema(conn)

Running query in 'sqlite:///wine.db'

Running query in 'sqlite:///wine.db'

Running query in 'sqlite:///wine.db'

Running query in 'sqlite:///wine.db'

Running query in 'sqlite:///wine.db'

...

Running query in 'sqlite:///wine.db'

Running query in 'sqlite:///wine.db'

Running query in 'sqlite:///wine.db'

Running query in 'sqlite:///wine.db'


Content of the database
CLIENTS:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
LOC:
  0: LIEU(TEXT)
  1: REGION(TEXT)
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
PRODUCERS:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
PRODUCTIONS:
  0: NV(NUM)
  1: NP(NUM)
  2: QTE(NUM)
SALES:
  0: NV(NUM)
  1: NB(NUM)
  2: QTE(NUM)
  3: DATES(NUM)
  4: LIEU(TEXT)
WINES1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
WINES2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)


___
# PART I: CREATE AND USE VIEWS

A view is a virtual table based on the result-set of an SQL statement. Views are stored in the database with an associated name.

Views are created following the syntax:

```sql
CREATE VIEW view_name AS
SELECT column1, column2.....
FROM table_name
WHERE [condition];
```

An useful command is:

```sql
DROP VIEW IF EXISTS view_name;
```


__Note:__ Use it with caution (only drop something if you are sure)

__1.1__ Create a view 'bons_buveurs' with the clients (buveurs) of type 'gros' or 'moyen'.

In [6]:
%%sql DROP VIEW IF EXISTS bons_buveurs;
CREATE VIEW bons_buveurs AS
SELECT *
FROM CLIENTS
WHERE TYPE = "moyen" OR TYPE = "gros";

Running query in 'sqlite:///wine.db'

In [7]:
# Test
%sql SELECT * FROM bons_buveurs ORDER BY nb;

Running query in 'sqlite:///wine.db'

NB,NOM,PRENOM,TYPE
2,Artaud,Antonin,moyen
3,Aron,Raymond,gros
4,Apollinaire,Guillaume,moyen
6,Arrabal,Fernando,gros
7,Anouilh,Jean,moyen
8,Aragon,Louis,gros
10,Andersen,Yann,gros
12,Bataille,Georges,moyen
13,Barthes,Roland,moyen
14,Bory,Jean Louis,gros


__1.2__ Create the view 'buveurs_asec' with clients (buveurs) who have not bought any wine.

In [8]:
%%sql DROP VIEW IF EXISTS buveurs_asec;
CREATE VIEW buveurs_asec AS
SELECT *
FROM CLIENTS
WHERE NB NOT IN
    (SELECT DISTINCT c.NB
    FROM WINES2 AS w
        JOIN SALES AS s ON w.NV = s.NV AND QTE > 0
        JOIN CLIENTS AS c ON s.NB = c.NB)
ORDER BY NB ASC;

Running query in 'sqlite:///wine.db'

In [9]:
# Test
%sql SELECT * FROM buveurs_asec ORDER BY nb;

Running query in 'sqlite:///wine.db'

NB,NOM,PRENOM,TYPE
11,Breton,Andre,petit
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
21,Corbiere,Tristan,petit
23,Corneille,Pierre,petit
25,Char,Rene,petit
27,Dumas,Alexandre,gros
29,Fournier,Alain,petit


__1.3__ Create the view 'buveurs_achats' complementary to the previous one.

In [10]:
%%sql DROP VIEW IF EXISTS buveurs_achats;
CREATE VIEW buveurs_achats AS
SELECT *
FROM CLIENTS
WHERE NB NOT IN
    (SELECT DISTINCT NB
    FROM buveurs_asec)
ORDER BY NB ASC;

Running query in 'sqlite:///wine.db'

In [11]:
# Test
%sql SELECT * FROM buveurs_achats ORDER BY nb;

Running query in 'sqlite:///wine.db'

NB,NOM,PRENOM,TYPE
1,Aristote,None,petit
2,Artaud,Antonin,moyen
3,Aron,Raymond,gros
4,Apollinaire,Guillaume,moyen
5,Audiberti,Jacques,petit
6,Arrabal,Fernando,gros
7,Anouilh,Jean,moyen
8,Aragon,Louis,gros
9,Ajar,Emile,petit
10,Andersen,Yann,gros


__1.4__ Create the view 'q83pl' (LIEU, CRU, QTE_BUE) that provides by LIEU and CRU the total quantities bought in 1983 by all the clients (buveurs).

In [12]:
%%sql DROP VIEW IF EXISTS q83pl;
CREATE VIEW q83pl AS
SELECT s.LIEU, w.CRU, SUM(s.QTE) AS QTE_BUE
FROM SALES AS s
    JOIN WINES2 AS w ON s.DATES LIKE '1983%' AND s.NV = w.NV
GROUP BY s.LIEU, w.CRU;

Running query in 'sqlite:///wine.db'

In [13]:
# Test
%sql SELECT * FROM q83pl ORDER BY LIEU ASC, CRU ASC;

Running query in 'sqlite:///wine.db'

LIEU,CRU,QTE_BUE
CAEN,Seyssel,3
LILLE,Pommard,5
LYON,Beaujolais Villages,10
LYON,Julienas,2
PARIS,Beaujolais Primeur,4
PARIS,Coteaux du Tricastin,1
PARIS,Pouilly Vinzelles,3
RENNES,Mercurey,1
ROCQUENCOURT,Beaujolais Villages,260
ROCQUENCOURT,Saint Amour,80


__1.5__ Can we define the same view with ascending order over the attribute QTE? Provide an explanation for your answer.

**Answer**: Ordering a view does not have any sense because a Table is not ordered, only the visualisation of it can be ordered. Still, sqlite allows to order the content when creating a view.

___
# PART II: UPDATE DATABASE CONTENT

The syntax for the Update statement is:

```sql
UPDATE table_name
SET column1 = value1, column2 = value2...., columnN = valueN
WHERE [condition];
```

The syntax for the Insert statement is:

```sql
INSERT INTO TABLE_NAME [(column1, column2, column3,...columnN)]  
VALUES (value1, value2, value3,...valueN);
```


Database updates are commited automatically in Jupyter/Python. _Transactions_ are an option to control and reverse changes. Additionally we can just reload a backup of the database (NOT an option in deployed systems)

__Note:__ Different to other Database Management Systems, SQLite views are read-only and so you may not execute a DELETE, INSERT or UPDATE statement on a view.

__2.1__ Create a table 'RBB' with the same schema as 'bons_buveurs' which contains the tuples selected from 'bons_buveurs'

In [14]:
%%sql DROP TABLE IF EXISTS RBB;
CREATE TABLE RBB AS
SELECT *
FROM bons_buveurs;

Running query in 'sqlite:///wine.db'

In [15]:
# Test
%sql SELECT * FROM RBB;

Running query in 'sqlite:///wine.db'

NB,NOM,PRENOM,TYPE
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
27,Dumas,Alexandre,gros
32,Eluard,Paul,moyen
35,Fromentin,Eugene,gros
39,Montesquieu,None,gros
42,Goethe,Johann Wolfgang von,moyen
43,Musset,Alfred de,gros


__2.2__ Update the table you used to create 'bons_buveurs': Change the 'type' to 'gros' if the total of quantities bought is over 100.

Find the instances to update (schema may be different from the one in your table)

In [16]:
%%sql
SELECT * FROM RBB
WHERE TYPE = "moyen" AND NB IN
    (SELECT NB
    FROM SALES
    GROUP BY NB
    HAVING SUM(QTE) > 99)

Running query in 'sqlite:///wine.db'

NB,NOM,PRENOM,TYPE
2,Artaud,Antonin,moyen


Update instances

In [17]:
%%sql
UPDATE RBB
SET TYPE = "gros"
WHERE NB IN 
    (SELECT NB FROM RBB
    WHERE TYPE = "moyen" AND NB IN
        (SELECT NB
        FROM SALES
        GROUP BY NB
        HAVING SUM(QTE) > 99));

Running query in 'sqlite:///wine.db'

1 rows affected.

__2.3__ Compare the content of _table_ 'RBB' and the _view_ 'bons_buveurs' after the update. What differences do you see? Explain

**Answer**: We can see that it changes the `TYPE` attribute for 1 instance.

In [18]:
%%sql
SELECT RBB.nb, RBB.NOM, RBB.PRENOM, RBB.TYPE AS UPDATE_TYPE, bb.TYPE AS PREVIOUS_TYPE
FROM RBB
JOIN bons_buveurs AS bb ON RBB.nb = bb.nb AND RBB.TYPE <> bb.TYPE;

Running query in 'sqlite:///wine.db'

NB,NOM,PRENOM,UPDATE_TYPE,PREVIOUS_TYPE
2,Artaud,Antonin,gros,moyen


__2.4__ Create a table 'RBA' with the same schema as 'buveurs_asec' which contains the tuples selected from 'buveurs_asec'

In [19]:
%%sql DROP TABLE IF EXISTS RBA;
CREATE TABLE RBA AS
SELECT *
FROM buveurs_asec;

Running query in 'sqlite:///wine.db'

In [20]:
# Test
%sql SELECT * FROM RBA

Running query in 'sqlite:///wine.db'

NB,NOM,PRENOM,TYPE
11,Breton,Andre,petit
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
21,Corbiere,Tristan,petit
23,Corneille,Pierre,petit
25,Char,Rene,petit
27,Dumas,Alexandre,gros
29,Fournier,Alain,petit


__2.5__ Insert a tuple (101, 'your last name', 'your first name', 'your type of purchases(petit, moyen, or gros)') to 'RBA'

In [21]:
%%sql
INSERT INTO RBA (NB, NOM, PRENOM, TYPE)
VALUES (101, 'Ternot', 'Benjamin', 'moyen');

Running query in 'sqlite:///wine.db'

1 rows affected.

In [22]:
# Test
%sql SELECT * FROM RBA ORDER BY NB DESC;

Running query in 'sqlite:///wine.db'

NB,NOM,PRENOM,TYPE
101,Ternot,Benjamin,moyen
99,Cesar,Jules,gros
98,Maturin,Robert,gros
95,Zorn,Fritz,moyen
93,Virgile,None,gros
91,Verlaine,Paul,gros
90,Vian,Boris,petit
89,Stendhal,None,petit
88,Shakespeare,William,petit
87,Segur,Comtesse de,gros


__2.6__ Compare the content of _table_ 'RBA' and the _view_ 'buveurs_asec'. What differences do you see? Explain

**Answer**: Well, in Table `RBA`, there is the instance for myself.

In [23]:
%%sql
SELECT *
FROM RBA
WHERE (NB, NOM, PRENOM, TYPE) NOT IN buveurs_asec;

Running query in 'sqlite:///wine.db'

NB,NOM,PRENOM,TYPE
101,Ternot,Benjamin,moyen


___
# PART III: Design the database schema for a marketplace

We consider the following scenario:

You are a developer for a game. In the game, there are several kinds of objects that the players can collect (e.g. magical flowers, gems, etc.). Players can also sell or buy objects from a marketplace that works in the following way: players declare some of the objects in their possession as “buyable”
with a price for each buyable object and then any player can buy items marked as buyable. When a player buys an item, the possession of the item is transferred to the buyer while the money corresponding to the item is transferred from the buyer to the seller. Note that, as long as the item is not sold, it still belongs to the player. Your goal is to
create a database for this marketplace.

The database schema you will develop needs to support players, the money each player has, the various types
of objects present in the game and how many objects of each kind each player has.Your database need to be able
to support the following kind of operations:

* Create a new type of object or add new players,
* Change the name of a type of objects or the name of a player,
* Attribute an object of a given type to a player,
* Increase or decrease the amount of money a player has,
* Retrieve the list of all the items that a player has,
* Compute the current balance of a player,
* Allow a player to mark one of their item as buyable with a given price,
* Allow a player to buy the cheapest item of a given type from the marketplace.

Ideally your schema should enforce the following constraints:

* All the objects that the players have correspond to some known type,
* Each object is possessed by exactly one player,
* Any single object cannot be marked twice as buyable, nor it can have two different prices (but two different objects of the same kind can have different prices),
* Any player has a positive amount of money.

__3.1__ Draw an entity-relationship model for the scenario.

_to answer, you can for example join a figure with the notebook or write the information related to the entity and relationship sets_

* Tables:

| Players |                                      |
| ------- | ------------------------------------ |
| ID      | Number id of the player (Primary Key) |
| NAME    | Name of the player                    |
| BALANCE | Amount of money the player has (Positive value)       |  

| Object_Types |                                  |
| ------------ | -------------------------------- |
| ID           | Number id of an object type  (Primary Key)    |
| NAME         | Name of an object type            |

| Objects |                                                 |
| ------- | ----------------------------------------------- |
| ID      | Number id of an object  (Primary Key)                         |
| TYPE_ID | Id of the Type of the object (Foreign Key referencing `Object_Types.ID`)       |
| OWNER_ID| Id of the player that possesses the object (Foreign Key referencing `Players.ID`) |  

| Market_Place |                                        |
| ------------ | -------------------------------------- |
| OBJECT_ID    | Id of the object for sale (Foreign Key referencing `Objects.ID`, Unique)  |
| PRICE        | Price for buying the object              |

* Entities:  

    * Player: Represents the players in the game.
        Attributes: ID (Primary Key), Name, Balance

    * Object_Type: Represents the various types of objects in the game.
        Attributes: ID (Primary Key), Name

    * Object: Represents the objects possessed by players.
        Attributes: ID (Primary Key), Type_ID (Foreign Key referencing `Object_Type.ID`), Owner_ID (Foreign Key referencing `Player.ID`)

    * Market_Place: Represents the marketplace where players can buy and sell objects.
        Attributes: Object_ID (Foreign Key referencing `Object.ID`), Price

* Relationships:

    * Possession, Objects-Players (Many to one) : Indicates the ownership relationship between players and objects.
        One player can possess many objects, but each object is possessed by only one player.

    * Object_Type, Objects-Object_Type (Many to one) : Indicates the type of an object.
        Each object belongs to a single specific object type.

    * Buy/Sell, Objects-Market_Place (Many to one): Indicates the transactional relationship between players in the marketplace.
        Players can buy and sell objects in the marketplace. Each object can be put in the marketplace but cannont be present in multiple instances

__3.2__ Deduce the relational scheme while precising the _primary_ and _foreign_ (if any) keys for each relation.

In SQL, that would give something like that:

In [24]:
%reload_ext sql
%sql sqlite:///game.db

%sql DROP TABLE IF EXISTS Players;
%sql DROP TABLE IF EXISTS Object_Types;
%sql DROP TABLE IF EXISTS Objects;
%sql DROP TABLE IF EXISTS Market_Place;

%sql CREATE TABLE Players (\
    ID INT PRIMARY KEY,\
    NAME VARCHAR(255),\
    BALANCE DECIMAL(10, 2) CHECK (BALANCE >= 0)\
);

%sql CREATE TABLE Object_Types (\
    ID INT PRIMARY KEY,\
    NAME VARCHAR(255)\
);

%sql CREATE TABLE Objects (\
    ID INT PRIMARY KEY,\
    TYPE_ID INT,\
    OWNER_ID INT,\
    FOREIGN KEY (TYPE_ID) REFERENCES Object_Types(ID),\
    FOREIGN KEY (OWNER_ID) REFERENCES Players(ID)\
);

%sql CREATE TABLE Market_Place (\
    OBJECT_ID INT,\
    PRICE DECIMAL(10, 2),\
    PRIMARY KEY (OBJECT_ID),\
    FOREIGN KEY (OBJECT_ID) REFERENCES Objects(ID)\
);

Running query in 'sqlite:///game.db'

Running query in 'sqlite:///game.db'

Running query in 'sqlite:///game.db'

Running query in 'sqlite:///game.db'

Running query in 'sqlite:///game.db'

Running query in 'sqlite:///game.db'

Running query in 'sqlite:///game.db'

Running query in 'sqlite:///game.db'

In [25]:
conn2 = sqlite3.connect('game.db')
print("Database schema:\n")
printSchema(conn2)

Database schema:

Market_Place:
  0: OBJECT_ID(INT) *1
  1: PRICE(DECIMAL(10, 2))
Object_Types:
  0: ID(INT) *1
  1: NAME(VARCHAR(255))
Objects:
  0: ID(INT) *1
  1: TYPE_ID(INT)
  2: OWNER_ID(INT)
Players:
  0: ID(INT) *1
  1: NAME(VARCHAR(255))
  2: BALANCE(DECIMAL(10, 2))
